<a href="https://colab.research.google.com/github/GnomeRapiere/Projets-Python/blob/main/TD2Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import hashlib
import binascii

####### PREMIERE PARTIE DU TD ###########

#programme récupérant une sortie via interaction avec l'utilisateur
def interactionterminal():
  print("Veuillez effectuez votre saisie ")
  sortie = input()
  return sortie

#génère un nombre aléatoire
def aleatoiresecurise():
  x = int(binascii.hexlify(os.urandom(16)),16)
  return x

x = aleatoiresecurise()
print(x)

#conversion de ce nombre en binaire 
binaire=str(format(x, "b"))
print("En binaire : "+binaire)

#on met ce nombre binaire dans le bon format (128 bits)

for i in range(128):
  if(len(binaire)<=128-1):
    binaire=str(((128-len(binaire)-2)*0))+binaire
print("En binaire 128 bits : "+ binaire)

#hachage en SHA256 pour récupérer les 4 derniers nombres
hex=hashlib.sha256(binaire.encode('utf-8')).hexdigest()
#print("SHA256: "+hex)
int_valeur = int(hex, base=16) #nombre en base 16
binaire_sha256=str(format(int_valeur, "b")) #nombre en binaire
#print(binaire_sha256)
binaire=binaire+binaire_sha256[0]+binaire_sha256[1]+binaire_sha256[2]+binaire_sha256[3] 
print(binaire) #nombre final codé sur 132 bits qu'on sépare en 11 packets.

#Maintenant on sépare ce nombre en 12 lots de 11 bits.

packets = []
packet = "" #packet au singulier correspond à l'élément (un lot de 11 bits)
j=0

for k in range(12):
  for i in range(11):
    packet = packet + binaire[i+j]
  packets.append(packet)
  packet = ""
  j= j+11
#La liste packets contient désormais tous nos packets.
print(packets)

packets2=[]
for s in range(len(packets)):
  packets2.append(int(packets[s],2)) 

#On reconvertit les nombres binaires pour obtenir les indices à chercher dans le dictionnaire BIP39
print(packets2)

#On fait correspondre nos indices aux mots du dictionnaire bip39 et on les implémente ensuite dans la liste seed.
seed = []
dicobip39 = []
file = open('/bip39.txt', "r")
for line in file:
  dicobip39.append(line)
file.close()

for i in range(len(packets2)):
  seed.append(dicobip39[packets2[i]])

#print(seed)

#On met la liste seed dans le bon format (en enlevant les \n)
for i in range(len(seed)):
  seed[i] = seed[i][:-1]

print(seed) #on a enfin notre seed dans le bon format !

#Cette fois on va importer une seed directement saisie par l'utilisateur via ligne de commande et vérifier qu'elle est correcte.

seed2=[]
seed3=[]

#D'abord l'utilisateur saisie manuellement les 8 mots de sa seed, ils sont ensuite reconvertis en binaire pour retrouver nos 12 paquets 
#de 11bits.

for y in range(12):
    ax=str(format(dicobip39.index(input("Saisir mot "+str(y)+" : ")+'\n'),"b"))
    seed2.append(ax)

#Suite à cela on rajoute les 0 manquants pour avoir le bon nombre de bits.
for o in range(12):
    string=seed2[o]
    if(len(string)<=11):
        string=str(((11-len(string))*str(0)))+string
        seed3.append(string)
    else:
        seed3.append(string)
print(seed2) #des paquets pas forcement de 11
print(seed3) #des paquets de 11 => seed3 est dans le format souhaité.

binaire_seed132=""
for h in range(len(seed3)):
    binaire_seed132=binaire_seed132+seed3[h]
print(binaire_seed132) #on retrouve ici le nombre en 132 bits

#il faut vérifier que les 4 derniers caracteres (checksum) correspondent aux 4 premiers caractères
#du resultat après le hachage du nombre en 128 bits.

checksum=binaire_seed132[128]+binaire_seed132[129]+binaire_seed132[130]+binaire_seed132[131]
print(checksum)
binaire_seed128=binaire_seed132[:-4] #on récupère le nombre amputé des 4 derniers caracteres pour pouvoir le hacher.
print(binaire_seed128)
hex_seed=hashlib.sha256(binaire_seed128.encode('utf-8')).hexdigest() #on hache
int_valeur_seed = int(hex_seed, base=16) #nombre en base 16
binaire_sha256_seed=str(format(int_valeur_seed, "b")) #nombre en binaire
binaire_sha256_checksum=binaire_sha256_seed[0]+binaire_sha256_seed[1]+binaire_sha256_seed[2]+binaire_sha256_seed[3]
print(binaire_sha256_checksum) 
#on compare nos 4 caractères aux 4 premiers après hachage pour vérifier si ils correspondent.
if(binaire_sha256_checksum==checksum):
    print("La seed saisie est bien valide.")
else:
  print("La seed saisie est invalide.")




  





173235732026500073916491275656490053978
En binaire : 10000010010100111111110100000110001000001111100001111100010001100100101001101000010001001111010111000001010100101111000101011010
En binaire 128 bits : 10000010010100111111110100000110001000001111100001111100010001100100101001101000010001001111010111000001010100101111000101011010
100000100101001111111101000001100010000011111000011111000100011001001010011010000100010011110101110000010101001011110001010110101101
['10000010010', '10011111111', '01000001100', '01000001111', '10000111110', '00100011001', '00101001101', '00001000100', '11110101110', '00001010100', '10111100010', '10110101101']
[1042, 1279, 524, 527, 1086, 281, 333, 68, 1966, 84, 1506, 1453]
['liquid', 'paper', 'dose', 'draft', 'march', 'case', 'claim', 'ancient', 'volume', 'appear', 'rough', 'remember']
Saisir mot 0 : liquid
Saisir mot 1 : paper
Saisir mot 2 : dose
Saisir mot 3 : draft
Saisir mot 4 : march
Saisir mot 5 : case
Saisir mot 6 : claim
Saisir mot 7 : ancient
Sais

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
#génère un nombre aléatoire
import os
import binascii
def aleatoiresecurise():
  x = int(binascii.hexlify(os.urandom(16)),16)
  return x

x = aleatoiresecurise()
print(x)

44064172506337978199024044592053051820
